In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pickle
from sklearn.datasets import fetch_lfw_pairs
from train_tensorflow.FaceNet import call_instance_FaceNet_with_last_ArcFace, convert_arcface_model_to_embedding,\
                                        convert_arcface_model_to_embeddingv2,call_instance_FaceNet_with_last_isDense,\
                                        convert_train_model_to_embedding
from scipy.spatial.distance import cosine,euclidean
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
from facenet_pytorch import MTCNN
from train_pytorch.detect_face import FaceDetector
from product.MyModel import MyModel


def main():
    # setup
    my_model = MyModel("ArcFace")
    lfw = fetch_lfw_pairs(subset='test', color=True, resize=1.0, slice_=(slice(0, 250), slice(0, 250)))
    pairs = lfw.pairs
    labels = lfw.target
    face_detector = FaceDetector()
    
    #Loop different threshold
    thresholds = np.arange(0,2.01,0.01)
    predictions = [[] for i in range(len(thresholds))]
    for i in tqdm(range(0, pairs.shape[0])):
        pair = pairs[i]
        img1 = face_detector.detect_one_face(pair[0], optimize_margin = True)
        img2 = face_detector.detect_one_face(pair[1], optimize_margin = True)
        prediction = my_model.verify(img1, img2, thresholds, distance_type = "euclidean") #this should return 1 for same person, 0 for different persons.
        for j in range(len(prediction)):
            predictions[j].append(prediction[j])
    scores = []
    for prediction in predictions:
        score = accuracy_score(labels, prediction)*100
        scores.append(score)
    for j in range(len(scores)):
        print(f"threshold {thresholds[j]} accuracy_score {scores[j]}")


main()

 31%|███▏      | 313/1000 [03:00<06:31,  1.75it/s]